In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import pandas as pd
import conceptlab as clab
import numpy as np

Load results

In [11]:
df = pd.read_csv('../../results/sweeps/wandb_kang_results.csv')

Drop unnecessary column

In [12]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

Inspect

In [13]:
df.head()

,model,dataset,analysis,input_dim,n_concepts,constants.seed,model.independent_training,B cells_ctrl_On_stim_Off_none_mmd_ratio,B cells_ctrl_On_stim_Off_none_post_ivn_r2_score,B cells_ctrl_On_stim_Off_none_pre_ivn_r2_score,...,CD8 T cells_ctrl_On_stim_Off_none_pre_ivn_r2_score,Megakaryocytes_ctrl_On_stim_Off_none_mmd_ratio,Megakaryocytes_ctrl_On_stim_Off_none_post_ivn_r2_score,Megakaryocytes_ctrl_On_stim_Off_none_pre_ivn_r2_score,FCGR3A+ Monocytes_ctrl_On_stim_Off_none_mmd_ratio,FCGR3A+ Monocytes_ctrl_On_stim_Off_none_post_ivn_r2_score,FCGR3A+ Monocytes_ctrl_On_stim_Off_none_pre_ivn_r2_score,NK cells_ctrl_On_stim_Off_none_mmd_ratio,NK cells_ctrl_On_stim_Off_none_post_ivn_r2_score,NK cells_ctrl_On_stim_Off_none_pre_ivn_r2_score
0,skip_cb_vae,kang,kang_b_cells,9224,9,13,True,0.263590,0.973941,0.887552,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,skip_cb_vae,kang,kang_b_cells,9224,9,42,True,0.210605,0.979048,0.889842,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,skip_cb_vae,kang,kang_b_cells,9224,9,1337,True,0.230054,0.977167,0.889950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,skip_cb_vae,kang,kang_cd14_monocytes,9224,9,1337,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,skip_cb_vae,kang,kang_cd4_t_cells,9224,9,13,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Define columns to collapse over

In [14]:
collapse_columns = ['constants.seed'] + [x for x in df.columns if '_mmd_ratio' in x]

Get mean and std over collapsed columns

In [15]:
mean_grouped = clab.utils.wandb.collapse_columns(df[collapse_columns + ['analysis']], collapse_columns=collapse_columns,)
std_grouped = clab.utils.wandb.collapse_columns(df[collapse_columns + ['analysis']], collapse_columns=collapse_columns, agg = 'std')

Drop unecessary columns (after collapse)

In [16]:
for dff in [mean_grouped, std_grouped]:
    dff.drop(columns = ['analysis','constants.seed'], inplace = True)

Reformat data

In [17]:
res_dict = dict()
for k in range(len(mean_grouped)):
    m_row = mean_grouped.iloc[k]
    s_row = std_grouped.iloc[k]
    mix = np.argmax(~m_row.isna())
    name = mean_grouped.columns[mix].split('_ctrl_')[0]
    res_dict[name] = str(m_row[mix].round(3)) + '$\pm$' + str(s_row[mix].round(3))

/tmp/ipykernel_794210/934171902.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  res_dict[name] = str(m_row[mix].round(3)) + '$\pm$' + str(s_row[mix].round(3))


Create Pretty Table for print

In [18]:
pretty_table = pd.DataFrame(pd.Series(res_dict))

In [19]:
pretty_table.columns = ['rMMD']
pretty_table.index.name = 'Cell Type'
pretty_table['outcome'] = np.where([float(x.split('.')[0]) >= 1 for x in pretty_table['rMMD'].values], '\\xmark', '\\cmark')

Print Table 3 in Paper

In [20]:
print(pretty_table.reset_index().to_latex(index=False))

\begin{tabular}{lll}
\toprule
Cell Type & rMMD & outcome \\
\midrule
B cells & 0.235$\pm$0.027 & \cmark \\
CD14+ Monocytes & 1.684$\pm$0.244 & \xmark \\
CD4 T cells & 0.751$\pm$0.256 & \cmark \\
CD8 T cells & 0.253$\pm$0.078 & \cmark \\
Dendritic cells & 0.197$\pm$0.014 & \cmark \\
FCGR3A+ Monocytes & 0.366$\pm$0.044 & \cmark \\
Megakaryocytes & 0.374$\pm$0.037 & \cmark \\
NK cells & 0.301$\pm$0.049 & \cmark \\
\bottomrule
\end{tabular}



Display of Table 3

In [21]:
pretty_table

,rMMD,outcome
Cell Type,,
B cells,0.235$\pm$0.027,\cmark
CD14+ Monocytes,1.684$\pm$0.244,\xmark
CD4 T cells,0.751$\pm$0.256,\cmark
CD8 T cells,0.253$\pm$0.078,\cmark
Dendritic cells,0.197$\pm$0.014,\cmark
FCGR3A+ Monocytes,0.366$\pm$0.044,\cmark
Megakaryocytes,0.374$\pm$0.037,\cmark
NK cells,0.301$\pm$0.049,\cmark
